In [1]:
import os, sys

from alphahelix_database_tools.cloud_database import CloudArticlesDatabase

In [5]:
article_db  = CloudArticlesDatabase("/Users/yahoo168/Desktop/資料庫_測試功能/MDB_config")
# article_db.save_shorts()
# article_db.save_shorts_summary()

In [ ]:
stock_ticker_list = ["AAPL", "GOOG", "MSFT", "OXY", "LAZR", "NVTS", "QCOM", "TSLA", "NET", "ON", "OXY", "TSM"]
bond_ticker_list = ['OXY', 'RITM', 'CXW', 'F', 'MO', 'BA']
pool_ticker_list = stock_ticker_list + bond_ticker_list

for ticker in pool_ticker_list:
    article_db.save_stock_report_review(ticker=ticker)

In [9]:
article_db.process_raw_stock_report()

[20240711 14:54:03 INFO    ] [SERVER][Data Process][共180篇non auto stock report待處理]
[20240711 14:54:03 INFO    ] [SERVER][Data Process][開始處理第0篇non auto stock report]
[nltk_data] Downloading package punkt to /Users/yahoo168/nltk_data...
[nltk_data]   Package punkt is already up-to-date!
[20240711 14:54:05 INFO    ] [LLM][preprocess]Barclays_Apple_Inc_AAPL_Preview_Expecting_In_line_March_Q_and_Potential_June_Q_Guide_down_on_Weak_Hardware_iPhone_Demand.pdf
[20240711 14:54:16 INFO    ] HTTP Request: POST https://api.openai.com/v1/chat/completions "HTTP/1.1 200 OK"
[20240711 14:54:22 INFO    ] HTTP Request: POST https://api.openai.com/v1/chat/completions "HTTP/1.1 200 OK"
[20240711 14:54:22 INFO    ] [SERVER][Data Process][開始處理第1篇non auto stock report]
[nltk_data] Downloading package punkt to /Users/yahoo168/nltk_data...
[nltk_data]   Package punkt is already up-to-date!
[20240711 14:54:23 INFO    ] [LLM][preprocess]Barclays_Apple_Inc_AAPL_Supplier_Tracker_Lower_Hardware_Builds_for_Feb_Month

In [ ]:
ticker = "MSFT"

stock_following_issue_meta_list = article_db.MDB_client["users"]["following_issues"].find({"tickers": ticker}, limit=10)
stock_following_issue_list = [meta["issue"] for meta in stock_following_issue_meta_list]
stock_following_issue_list

In [ ]:
processed_list = list(article_db.MDB_client["preprocessed_content"]["stock_report"].find())

In [ ]:
for meta in processed_list:
    blob_name = meta["blob_name"]
    raw_meta = article_db.MDB_client["raw_content"]["raw_stock_report_non_auto"].update_one({"blob_name": blob_name}, {"$set": {"is_processed": True}})
    print(raw_meta)
    

In [ ]:
# 查询条件：查找所有不包含is_processed字段的文档
query = {"is_processed": {"$exists": False}}

# 更新操作：将is_processed字段设置为false
update = {"$set": {"is_processed": False}}

# 批量更新文档
result = article_db.MDB_client["raw_content"]["raw_stock_report_non_auto"].update_many(query, update)

In [8]:
update = {"$rename": {"date": "data_timestamp"}}
# 批量更新文档
result = article_db.MDB_client["raw_content"]["raw_stock_report_non_auto"].update_many({}, update)
result = article_db.MDB_client["preprocessed_content"]["stock_report"].update_many({}, update)

In [6]:
unprocessed_stock_report_meta_list = list(article_db.MDB_client["raw_content"]["raw_stock_report_non_auto"].find({"is_processed": False, ""}))
len(unprocessed_stock_report_meta_list)

180